## Integral

In [4]:
from __future__ import annotations

from dataclasses import dataclass
from typing import Optional, Union, Dict, Any

import sympy as sp
from sympy.parsing.sympy_parser import (
    parse_expr,
    standard_transformations,
    implicit_multiplication_application,
    convert_xor,
)


@dataclass
class IntegralResult:
    expr: sp.Expr              # parsed expression
    integral: sp.Expr          # symbolic integral
    definite_value: Optional[sp.Expr]  # definite integral result if bounds provided


def integrate_from_string(
    func_str: str,
    var: str = "x",
    lower: Optional[Union[int, float, str]] = None,
    upper: Optional[Union[int, float, str]] = None,
    extra_symbols: Optional[Dict[str, Any]] = None,
) -> IntegralResult:
    """
    Parse a function string and integrate it with respect to `var`.

    Parameters
    ----------
    func_str : str
        Function as a string, e.g. "sin(x) + x**2" or "2x + 1".
    var : str
        Variable name, default "x".
    lower, upper : optional
        If both are provided, compute definite integral from lower to upper.
        Can be numbers or strings like "pi/2".
    extra_symbols : dict, optional
        Extra allowed symbols, e.g. {"a": sp.Symbol("a")} if your function uses parameters.

    Returns
    -------
    IntegralResult
        Contains parsed expression, symbolic integral, and optional definite value.
    """

    # Define the integration variable
    x = sp.Symbol(var)

    # Transformations:
    # - convert_xor: allow "^" to mean exponent (so "x^2" works)
    # - implicit multiplication: allow "2x" to mean "2*x"
    transformations = (
        standard_transformations
        + (implicit_multiplication_application, convert_xor)
    )

    # Whitelist names that parsing is allowed to use
    local_dict = {
        var: x,
        "pi": sp.pi,
        "E": sp.E,
        "e": sp.E,
        # common functions
        "sin": sp.sin,
        "cos": sp.cos,
        "tan": sp.tan,
        "asin": sp.asin,
        "acos": sp.acos,
        "atan": sp.atan,
        "sinh": sp.sinh,
        "cosh": sp.cosh,
        "tanh": sp.tanh,
        "exp": sp.exp,
        "log": sp.log,   # natural log
        "ln": sp.log,
        "sqrt": sp.sqrt,
        "abs": sp.Abs,
    }

    if extra_symbols:
        local_dict.update(extra_symbols)

    # Parse safely (no eval)
    expr = parse_expr(func_str, local_dict=local_dict, transformations=transformations)

    # Symbolic integral
    integral_expr = sp.integrate(expr, x)

    definite_value = None
    if lower is not None and upper is not None:
        # allow bounds like "pi/2"
        lo = parse_expr(str(lower), local_dict=local_dict, transformations=transformations)
        hi = parse_expr(str(upper), local_dict=local_dict, transformations=transformations)
        definite_value = sp.integrate(expr, (x, lo, hi))

    return IntegralResult(expr=expr, integral=integral_expr, definite_value=definite_value)



In [5]:
integrate_from_string("(e**(x**2) - 1)/(e-1)", lower=0, upper=1)

IntegralResult(expr=(exp(x**2) - 1)/(-1 + E), integral=(-x + sqrt(pi)*erfi(x)/2)/(-1 + E), definite_value=(-1 + sqrt(pi)*erfi(1)/2)/(-1 + E))

In [8]:
resx  = integrate_from_string("sin(x) * (2* x * e^(x^2))/(e-1)", lower=0, upper=1)
resx

IntegralResult(expr=2*x*exp(x**2)*sin(x)/(-1 + E), integral=2*Integral(x*exp(x**2)*sin(x), x)/(-1 + E), definite_value=2*Integral(x*exp(x**2)*sin(x), (x, 0, 1))/(-1 + E))

In [9]:
resx.definite_value.evalf()

0.652774581828748

## Derivative

In [1]:
from __future__ import annotations

from dataclasses import dataclass
from typing import Optional, Union, Dict, Any

import sympy as sp
from sympy.parsing.sympy_parser import (
    parse_expr,
    standard_transformations,
    implicit_multiplication_application,
    convert_xor,
)


@dataclass
class DerivativeResult:
    expr: sp.Expr                 # parsed expression
    derivative: sp.Expr           # symbolic derivative
    evaluated_value: Optional[sp.Expr]  # derivative evaluated at a point (if provided)


def derivative_from_string(
    func_str: str,
    var: str = "x",
    at: Optional[Union[int, float, str]] = None,
    order: int = 1,
    extra_symbols: Optional[Dict[str, Any]] = None,
) -> DerivativeResult:
    """
    Parse a function string and differentiate it with respect to `var`.

    Parameters
    ----------
    func_str : str
        Function as a string, e.g. "sin(x) + x**2" or "2x + 1".
    var : str
        Variable name, default "x".
    at : optional
        If provided, evaluate the derivative at this point (number or string like "pi/2").
    order : int
        Derivative order (1 = first derivative, 2 = second, etc.)
    extra_symbols : dict, optional
        Extra allowed symbols, e.g. {"a": sp.Symbol("a")} if your function uses parameters.

    Returns
    -------
    DerivativeResult
        Contains parsed expression, symbolic derivative, and optional evaluated value.
    """

    if order < 1:
        raise ValueError("order must be >= 1")

    # Define the differentiation variable
    x = sp.Symbol(var)

    transformations = (
        standard_transformations
        + (implicit_multiplication_application, convert_xor)
    )

    # Whitelist allowed names (safe parsing; no eval)
    local_dict = {
        var: x,
        "pi": sp.pi,
        "E": sp.E,
        "e": sp.E,
        # common functions
        "sin": sp.sin,
        "cos": sp.cos,
        "tan": sp.tan,
        "asin": sp.asin,
        "acos": sp.acos,
        "atan": sp.atan,
        "sinh": sp.sinh,
        "cosh": sp.cosh,
        "tanh": sp.tanh,
        "exp": sp.exp,
        "log": sp.log,
        "ln": sp.log,
        "sqrt": sp.sqrt,
        "abs": sp.Abs,
    }

    if extra_symbols:
        local_dict.update(extra_symbols)

    expr = parse_expr(func_str, local_dict=local_dict, transformations=transformations)

    # Symbolic derivative
    deriv = sp.diff(expr, x, order)

    evaluated_value = None
    if at is not None:
        at_expr = parse_expr(str(at), local_dict=local_dict, transformations=transformations)
        evaluated_value = deriv.subs(x, at_expr)

    return DerivativeResult(expr=expr, derivative=deriv, evaluated_value=evaluated_value)


In [2]:
derivative_from_string("sin(x^2) + x")

DerivativeResult(expr=x + sin(x**2), derivative=2*x*cos(x**2) + 1, evaluated_value=None)